In [ ]:
import glob
import cv2
import os
from ultralytics import YOLO
import numpy as np
import logging

# Configuration
MODEL_PATH = "MODEL_PATH"

VIDEO_FILES = [
    "VIDEO_PATH_1.mp4",
    "VIDEO_PATH_2.mp4",
    "...."
]

OUTPUT_BASE = "0411"

CONFIDENCE_THRESHOLD = 0.50

TARGET_CLASSES = None  # Set to None to detect all classes, or specify list like [0, 1, 2]
    # Configuration
MODEL_PATH = "MODEL_PATH"

VIDEO_FILES = [
    "VIDEO_PATH_1.mp4",
    "VIDEO_PATH_2.mp4",
    "...."
]

OUTPUT_BASE = "OUTPUT_FOLDER"

CONFIDENCE_THRESHOLD = 0.50

TARGET_CLASSES = None  # Set to None to detect all classes, or specify list like [0, 1, 2]

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class ObjectDetector:
    def __init__(self, model_path, conf_threshold=0.05):
        """
        Initialize the detector with model path and confidence threshold
        """
        self.model = YOLO(model_path)
        self.conf_threshold = conf_threshold
        
    def process_video(self, video_path, output_folder, target_classes=None):
        """
        Process video and save frames with detections.  
        Yeni sürüm: Model herhangi bir nesne tespit etmese bile her 4. kare görüntü ve etiketi (boş dosya) olarak kaydedilir.
        """
        # Create output folders
        frame_folder = os.path.join(output_folder, "images")
        annotation_folder = os.path.join(output_folder, "labels")
        annotated_frame_folder = os.path.join(output_folder, "annotateds")

        os.makedirs(frame_folder, exist_ok=True)
        os.makedirs(annotation_folder, exist_ok=True)
        os.makedirs(annotated_frame_folder, exist_ok=True)
        
        # Open video
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            raise ValueError(f"Error opening video file: {video_path}")
            
        frame_count = 0
        total_detections = 0
        
        try:
            i = 0
            while cap.isOpened():
                success, frame = cap.read()
                if not success:
                    break
                    
                i += 1

                frame_count += 1

                if i % 3 != 0: 
                    continue
                
                logger.info(f"Processing frame {frame_count}")
                
                # Run detection
                results = self.model.predict(
                    frame,
                    conf=self.conf_threshold,
                    iou=0,
                    classes=target_classes,
                    verbose=False
                )
                base_name = os.path.basename(video_path).split(".")[0]
                # Yalın kareyi her durumda kaydet
                frame_path = os.path.join(frame_folder, f"{base_name}_0411_{frame_count:06d}.jpg")
                cv2.imwrite(frame_path, frame)

                # İlgili etiket dosya yolu
                annotation_path = os.path.join(annotation_folder, f"{base_name}_0411_{frame_count:06d}.txt")

                # İşlenen her sonuç için (genelde tek bir sonuç döner)
                has_detections = False
                for result in results:
                    num_detections = len(result.boxes)
                    total_detections += num_detections

                    if num_detections > 0:
                        has_detections = True
                        # Etiket dosyasını kaydet
                        self._save_annotations(
                            result.boxes,
                            frame.shape,
                            annotation_path
                        )

                        # Log detection details
                        classes = result.boxes.cls.cpu().numpy()
                        confidences = result.boxes.conf.cpu().numpy()
                        logger.info(f"Frame {frame_count}: Found {num_detections} objects")
                        logger.info(f"Classes detected: {classes}")
                        logger.info(f"Confidence scores: {confidences}")

                        # Draw boxes on frame (optional)
                        annotated_frame = self._draw_boxes(frame, result.boxes)
                        cv2.imwrite(
                            os.path.join(
                                annotated_frame_folder,
                                f"0508_{frame_count:06d}_annotated.jpg"
                            ),
                            annotated_frame,
                        )

                # Eğer hiçbir nesne tespit edilmediyse, boş etiket dosyası oluştur
                if not has_detections:
                    open(annotation_path, "w").close()
                    logger.info(f"Frame {frame_count}: No detections (empty label saved)")
                
        finally:
            cap.release()
            
        logger.info(f"Processing complete. Total frames: {frame_count}")
        logger.info(f"Total detections: {total_detections}")
        return frame_count, total_detections
    
    def _save_annotations(self, boxes, image_shape, output_path):
        """
        Save detections in YOLO format
        """
        img_height, img_width = image_shape[:2]
        
        with open(output_path, 'w') as f:
            for box in boxes:
                # Get box coordinates
                x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)
                
                # Convert to YOLO format
                width = x2 - x1
                height = y2 - y1
                center_x = x1 + width / 2
                center_y = y1 + height / 2
                
                # Normalize coordinates
                center_x /= img_width
                center_y /= img_height
                width /= img_width
                height /= img_height
                
                # Write to file
                class_id = int(box.cls.item())
                f.write(f"{class_id} {center_x:.6f} {center_y:.6f} {width:.6f} {height:.6f}\n")
    
    def _draw_boxes(self, frame, boxes):
        """
        Draw detection boxes on frame
        """
        annotated_frame = frame.copy()
        
        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)
            conf = float(box.conf)
            class_id = int(box.cls.item())
            
            # Draw box
            cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            
            # Add label
            label = f"Class {class_id}: {conf:.2f}"
            cv2.putText(
                annotated_frame,
                label,
                (x1, y1 - 10),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.5,
                (0, 255, 0),
                2,
            )
            
        return annotated_frame

def main():

    # Initialize detector
    detector = ObjectDetector(MODEL_PATH, CONFIDENCE_THRESHOLD)

    # Process each video
    for video_file in VIDEO_FILES:
        logger.info(f"Processing video: {video_file}")
        
        # Create output folder for this video
        video_name = os.path.splitext(os.path.basename(video_file))[0]
        output_folder = os.path.join(OUTPUT_BASE, video_name)
        os.makedirs(output_folder, exist_ok=True)
        
        # Process video
        try:
            frames, detections = detector.process_video(
                video_file,
                output_folder,
                target_classes=TARGET_CLASSES,
            )
            logger.info(f"Completed processing {video_file}")
            logger.info(f"Processed {frames} frames with {detections} total detections")
            
        except Exception as e:
            logger.error(f"Error processing {video_file}: {str(e)}")

if __name__ == "__main__":
    main()

Bitti ✅ Kaydedilen kare sayısı: 656
